# Dynamic Evaluation with trace collection

* Difference with **static** evaluation: dynamic evaluation add trace collection on real GPU devices.
* Usage:
  * On Colab, Select CPU as Runtime, Click `Runtime -> Run All`
  * On other environment, first create a virturalenv and `Select Kernel` to use the virtualenv kernel, then click `Runtime -> Run All`
* Hardware Requirement: a NVIDIA GPU with the CUDA driver installed. Please note:
  1. The choice of hardware will significantly affect results. Please consider using A100, the same hardware we used in the paper.
  2. Please make sure no other workload is executing on the same GPU.
* Software Requirement: NEUTRINO system only depends on GNU toolchain (gcc, file, git, nm), CUDA toolchain (cuobjdump, ptxas) and Python 3.12 (pip, toml). But evaluation workload needs a PTX- included build of PyTorch and CUTLASS. We package the dependency checking and installation in prepare_env.py for one-click installation.
* Disk Space Requirement: Please arrange at least 10GB for collecting traces.

Expected results:
Dynamic evaluation on customized traces is expected to produce similar results, i.e., similar numbers or figure shapes. And if you encountered any problems, please contact us through HotCRP.

Notes:
Each following section can be executed independently after downloading the code and setting up the environment.

## 0. Prepare Environment

Prepare environment takes two steps:
1. Download the packaged code from anonymoused R2 object storage
2. Prepare the environment with [`test_env.py`](https://github.com/neutrino-gpu/neutrino/blob/artifact/artifact/prepare_env.py)

In [ ]:
!wget https://pub-eef24bf0aa5b4950860ea28dfbe39d8c.r2.dev/dynamic.zip
!unzip dynamic.zip

In [ ]:
import sys
!wget https://github.com/neutrino-gpu/neutrino/blob/artifact/artifact/prepare_env.py
!{sys.executable} prepare_env.py

## 1. Block Scheduling Cost

> We recommend you start with this as Hello World Example

* Correspondence to paper: Sec. 4.5
* Notes: Please complete this in "Kick-the-tires" run. This help justifying correct environment setup.

In [ ]:
%%writefile block_sched/memset.py
import torch
torch.zeros((4096,4096), dtype=torch.float16, device="cuda")

In [ ]:
# Then run the toolkit to check the result
!neutrino -p block_sched --tracedir block_sched python block_sched/raw.py

#### Block Scheduling Cost Optimization

As stated in Sec.4.5, there's two way to optimize the cost and gain performance improvement:
1. CUDA Driver's `memset` API
2. Persistent Kernel with Triton, written by Cursor backed by GPT-4o

In [ ]:
# First let's measure the time of raw implementation via Nsight System
!nsys nvprof python block_sched/raw.py

In [ ]:
%%writefile block_sched/memset.py
# Code for using CUDA Driver's memset
from cuda.bindings import driver
import torch

def _cudaGetErrorEnum(error):
    if isinstance(error, driver.CUresult):
        err, name = driver.cuGetErrorName(error)
        return name if err == driver.CUresult.CUDA_SUCCESS else "<unknown>"
    else:
        raise RuntimeError('Unknown error type: {}'.format(error))

def checkCudaErrors(result):
    if result[0].value:
        raise RuntimeError("CUDA error code={}({})".format(result[0].value, _cudaGetErrorEnum(result[0])))
    if len(result) == 1:
        return None
    elif len(result) == 2:
        return result[1]
    else:
        return result[1:]

# Initialize CUDA Driver API
checkCudaErrors(driver.cuInit(0))

# Retrieve handle for device 0
cuDevice = checkCudaErrors(driver.cuDeviceGet(0))

for _ in range(100): # Driver API is not stable, need more runs to get correct value
    a = torch.empty((4096, 4096), dtype=torch.float16, device="cuda")
    checkCudaErrors(driver.cuMemsetD16(a.data_ptr(), 0, 4096*4096))

In [ ]:
!nsys nvprof python block_sched/raw.py

In [ ]:
%%writefile block_sched/persist.py
# Persistent Kernel Implementation
# Acknowledgement: This is written by Cursor backed by GPT-4o
import torch
import triton
import triton.language as tl

@triton.jit
def zero_init_kernel_persistent(
    output_ptr,
    numel,
    BLOCK_SIZE: tl.constexpr,
    NUM_SMS: tl.constexpr
):
    # Get program ID
    start_pid = tl.program_id(axis=0)
    # Calculate number of blocks needed
    num_blocks = tl.cdiv(numel, BLOCK_SIZE)
    # Calculate blocks per SM
    blocks_per_sm = num_blocks // NUM_SMS
    if start_pid < num_blocks % NUM_SMS:
        blocks_per_sm += 1
    # Initialize block ID
    block_id = start_pid - NUM_SMS

    # Process multiple blocks per SM
    for _ in range(blocks_per_sm):
        block_id += NUM_SMS
        # Calculate offsets for this block
        offsets = block_id * BLOCK_SIZE + tl.arange(0, BLOCK_SIZE)
        # Create mask for valid elements
        mask = offsets < numel
        # Store zeros
        tl.store(output_ptr + offsets, tl.zeros([BLOCK_SIZE], dtype=tl.float16), mask=mask)

def zero_init_persistent(x):
    # Get total number of elements
    numel = x.numel()
    # Get number of SMs
    NUM_SMS = torch.cuda.get_device_properties("cuda").multi_processor_count
    # Configure BLOCK_SIZE still at 128
    BLOCK_SIZE = 128
    # Launch kernel
    grid = lambda META: (min(NUM_SMS, triton.cdiv(numel, META['BLOCK_SIZE'])),)
    zero_init_kernel_persistent[grid](
        x,
        numel,
        BLOCK_SIZE=BLOCK_SIZE,
        NUM_SMS=NUM_SMS,
        num_warps=8
    )

# NOTE replace with torch.empty to only allocate without initialize
a = torch.empty((4096, 4096), dtype=torch.float16, device="cuda")
for _ in range(100):
    zero_init_persistent(a)

In [ ]:
# To measure the time, we can use the Nsight System
!nsys nvprof python block_sched/persist.py

## 2. Densified Memory Access Timeline (DMAT) Plot

* Correspondence to paper: Fig.1, Fig.10(A/B/C), in total 4 figures presented
* Hardware: A100 GPU
* Notes:
  1. DMAT takes time, please be patient
  2. Each run of will yield slightly different results
  3. DMAT varies on different hardware, you will need to use an RTX3080 to have exactly the same results as presented
* Others: We will use two [Jupyter Magic](https://ipython.readthedocs.io/en/stable/interactive/magics.html#) to help evaluation
  * `%%writefile` will write the cell to path, *noted that the code won't be executed*, we use it to save code as Neutrino is a CLI tool.
  * `%%capture` will capture the `stdout` of the cell, we use it to read the path of generated DMAT Plot for display.

In [ ]:
%%writefile dmat/common.py
# Common Kernel of Fig.1 and Fig.10A
import torch
import triton
import triton.language as tl
from neutrino.utils.tensortrace import TensorTrace

@triton.jit
def _attn_fwd_inner(acc, l_i, m_i, q,  #
                    K_block_ptr, V_block_ptr,  #
                    start_m, qk_scale,  #
                    BLOCK_M: tl.constexpr, HEAD_DIM: tl.constexpr, BLOCK_N: tl.constexpr,  #
                    STAGE: tl.constexpr, offs_m: tl.constexpr, offs_n: tl.constexpr,  #
                    N_CTX: tl.constexpr, fp8_v: tl.constexpr):
    # range of values handled by this stage
    if STAGE == 1:
        lo, hi = 0, start_m * BLOCK_M
    elif STAGE == 2:
        lo, hi = start_m * BLOCK_M, (start_m + 1) * BLOCK_M
        lo = tl.multiple_of(lo, BLOCK_M)
    # causal = False
    else:
        lo, hi = 0, N_CTX
    K_block_ptr = tl.advance(K_block_ptr, (0, lo))
    V_block_ptr = tl.advance(V_block_ptr, (lo, 0))
    # loop over k, v and update accumulator
    for start_n in range(lo, hi, BLOCK_N):
        start_n = tl.multiple_of(start_n, BLOCK_N)
        # -- compute qk ----
        k = tl.load(K_block_ptr)
        qk = tl.dot(q, k)
        if STAGE == 2:
            mask = offs_m[:, None] >= (start_n + offs_n[None, :])
            qk = qk * qk_scale + tl.where(mask, 0, -1.0e6)
            m_ij = tl.maximum(m_i, tl.max(qk, 1))
            qk -= m_ij[:, None]
        else:
            m_ij = tl.maximum(m_i, tl.max(qk, 1) * qk_scale)
            qk = qk * qk_scale - m_ij[:, None]
        p = tl.math.exp2(qk)
        l_ij = tl.sum(p, 1)
        # -- update m_i and l_i
        alpha = tl.math.exp2(m_i - m_ij)
        l_i = l_i * alpha + l_ij
        # -- update output accumulator --
        acc = acc * alpha[:, None]
        # update acc
        v = tl.load(V_block_ptr)
        if fp8_v:
            p = p.to(tl.float8e5)
        else:
            p = p.to(tl.float16)
        acc = tl.dot(p, v, acc)
        # update m_i and l_i
        m_i = m_ij
        V_block_ptr = tl.advance(V_block_ptr, (BLOCK_N, 0))
        K_block_ptr = tl.advance(K_block_ptr, (0, BLOCK_N))
    return acc, l_i, m_i

@triton.jit
def _attn_fwd(Q, K, V, sm_scale, M, Out,  #
              stride_qz, stride_qh, stride_qm, stride_qk,  #
              stride_kz, stride_kh, stride_kn, stride_kk,  #
              stride_vz, stride_vh, stride_vk, stride_vn,  #
              stride_oz, stride_oh, stride_om, stride_on,  #
              Z, H, N_CTX,  #
              HEAD_DIM: tl.constexpr,  #
              BLOCK_M: tl.constexpr,  #
              BLOCK_N: tl.constexpr,  #
              STAGE: tl.constexpr  #
              ):
    tl.static_assert(BLOCK_N <= HEAD_DIM)
    start_m = tl.program_id(0)
    off_hz = tl.program_id(1)
    off_z = off_hz // H
    off_h = off_hz % H
    qvk_offset = off_z.to(tl.int64) * stride_qz + off_h.to(tl.int64) * stride_qh

    # block pointers
    Q_block_ptr = tl.make_block_ptr(
        base=Q + qvk_offset,
        shape=(N_CTX, HEAD_DIM),
        strides=(stride_qm, stride_qk),
        offsets=(start_m * BLOCK_M, 0),
        block_shape=(BLOCK_M, HEAD_DIM),
        order=(1, 0),
    )
    v_order: tl.constexpr = (0, 1) if V.dtype.element_ty == tl.float8e5 else (1, 0)
    V_block_ptr = tl.make_block_ptr(
        base=V + qvk_offset,
        shape=(N_CTX, HEAD_DIM),
        strides=(stride_vk, stride_vn),
        offsets=(0, 0),
        block_shape=(BLOCK_N, HEAD_DIM),
        order=v_order,
    )
    K_block_ptr = tl.make_block_ptr(
        base=K + qvk_offset,
        shape=(HEAD_DIM, N_CTX),
        strides=(stride_kk, stride_kn),
        offsets=(0, 0),
        block_shape=(HEAD_DIM, BLOCK_N),
        order=(0, 1),
    )
    O_block_ptr = tl.make_block_ptr(
        base=Out + qvk_offset,
        shape=(N_CTX, HEAD_DIM),
        strides=(stride_om, stride_on),
        offsets=(start_m * BLOCK_M, 0),
        block_shape=(BLOCK_M, HEAD_DIM),
        order=(1, 0),
    )
    # initialize offsets
    offs_m = start_m * BLOCK_M + tl.arange(0, BLOCK_M)
    offs_n = tl.arange(0, BLOCK_N)
    # initialize pointer to m and l
    m_i = tl.zeros([BLOCK_M], dtype=tl.float32) - float("inf")
    l_i = tl.zeros([BLOCK_M], dtype=tl.float32) + 1.0
    acc = tl.zeros([BLOCK_M, HEAD_DIM], dtype=tl.float32)
    # load scales
    qk_scale = sm_scale
    qk_scale *= 1.44269504  # 1/log(2)
    # load q: it will stay in SRAM throughout
    q = tl.load(Q_block_ptr)
    # stage 1: off-band
    # For causal = True, STAGE = 3 and _attn_fwd_inner gets 1 as its STAGE
    # For causal = False, STAGE = 1, and _attn_fwd_inner gets 3 as its STAGE
    if STAGE & 1:
        acc, l_i, m_i = _attn_fwd_inner(acc, l_i, m_i, q, K_block_ptr, V_block_ptr,  #
                                        start_m, qk_scale,  #
                                        BLOCK_M, HEAD_DIM, BLOCK_N,  #
                                        4 - STAGE, offs_m, offs_n, N_CTX, V.dtype.element_ty == tl.float8e5  #
                                        )
    # stage 2: on-band
    if STAGE & 2:
        # barrier makes it easier for compielr to schedule the
        # two loops independently
        acc, l_i, m_i = _attn_fwd_inner(acc, l_i, m_i, q, K_block_ptr, V_block_ptr,  #
                                        start_m, qk_scale,  #
                                        BLOCK_M, HEAD_DIM, BLOCK_N,  #
                                        2, offs_m, offs_n, N_CTX, V.dtype.element_ty == tl.float8e5  #
                                        )
    # epilogue
    m_i += tl.math.log2(l_i)
    acc = acc / l_i[:, None]
    m_ptrs = M + off_hz * N_CTX + offs_m
    tl.store(m_ptrs, m_i)
    tl.store(O_block_ptr, acc.to(Out.type.element_ty))

def forward(q, k, v, causal, sm_scale, BLOCK_M, BLOCK_N, NUM_STAGES, NUM_WARPS):
    # shape constraints
    HEAD_DIM_Q, HEAD_DIM_K = q.shape[-1], k.shape[-1]
    # when v is in float8_e5m2 it is transposed.
    HEAD_DIM_V = v.shape[-1]
    assert HEAD_DIM_Q == HEAD_DIM_K and HEAD_DIM_K == HEAD_DIM_V
    assert HEAD_DIM_K in {16, 32, 64, 128, 256}
    o = torch.empty_like(q)
    stage = 3 if causal else 1
    extra_kern_args = {}

    grid = (triton.cdiv(q.shape[2], BLOCK_M), q.shape[0] * q.shape[1], 1)
    M = torch.empty((q.shape[0], q.shape[1], q.shape[2]), device=q.device, dtype=torch.float32)

    # Actually prevent triton.autotune to take manual control
    config = triton.Config({}, num_stages=NUM_STAGES, num_warps=NUM_WARPS)
    _attn_fwd_tuned = triton.autotune(configs=[config, ], key=[])(_attn_fwd)

    _attn_fwd_tuned[grid](
            q, k, v, sm_scale, M, o,  #
            q.stride(0), q.stride(1), q.stride(2), q.stride(3),  #
            k.stride(0), k.stride(1), k.stride(2), k.stride(3),  #
            v.stride(0), v.stride(1), v.stride(2), v.stride(3),  #
            o.stride(0), o.stride(1), o.stride(2), o.stride(3),  #
            q.shape[0], q.shape[1],  #
            N_CTX=q.shape[2],  #
            HEAD_DIM=HEAD_DIM_K,  #
            BLOCK_M=BLOCK_M,
            BLOCK_N=BLOCK_N
            STAGE=stage,  #
            **extra_kern_args)
    return q, k, v, o, M, sm_scale, HEAD_DIM_K

In [ ]:
%%writefile dmat/fig1.py
# Fig 1A
import torch
from neutrino.utils.tensortrace import TensorTrace
from common import forward # defined above

BLOCK_M, BLOCK_N, NUM_STAGES, NUM_WARPS = 128, 64, 2, 8

torch.manual_seed(42)

with TensorTrace() as t:
    shape = (4, 32, 4096, 64) # batch_size, head_num, seq_len, head_dim
    q = torch.empty(shape, dtype=torch.float16, device="cuda").normal_(mean=0.0, std=0.5)
    k = torch.empty(shape, dtype=torch.float16, device="cuda").normal_(mean=0.0, std=0.5)
    v = torch.empty(shape, dtype=torch.float16, device="cuda").normal_(mean=0.0, std=0.5)
    # dout = torch.randn_like(q) # a random gradient
    causal = False
    sm_scale = 0.5

    forward(q, k, v, causal, sm_scale, BLOCK_M, BLOCK_N, NUM_STAGES, NUM_WARPS)

In [ ]:
%%capture cap --no-stderr # capture the stdout of this cell containing path to dmat.png
!neutrino -t dmat -k attn python dmat/fig1.py # our kernel has attn keyword in naming for matching

In [ ]:
Image(cap.stdout)

In [ ]:
%%writefile dmat/fig10a.py
# Fig. 10A code
import torch
from neutrino.utils.tensortrace import TensorTrace
from common import forward # defined above

BLOCK_M, BLOCK_N, NUM_STAGES, NUM_WARPS = 64, 64, 2, 4

torch.manual_seed(42)

with TensorTrace() as t:
    shape = (4, 32, 4096, 64) # batch_size, head_num, seq_len, head_dim
    q = torch.empty(shape, dtype=torch.float16, device="cuda").normal_(mean=0.0, std=0.5)
    k = torch.empty(shape, dtype=torch.float16, device="cuda").normal_(mean=0.0, std=0.5)
    v = torch.empty(shape, dtype=torch.float16, device="cuda").normal_(mean=0.0, std=0.5)
    # dout = torch.randn_like(q) # a random gradient
    causal = False
    sm_scale = 0.5

    forward(q, k, v, causal, sm_scale, BLOCK_M, BLOCK_N, NUM_STAGES, NUM_WARPS)

In [ ]:
%%capture cap --no-stderr # capture the stdout of this cell containing path to dmat.png
!neutrino -t dmat -k attn python dmat/fig10a.py

In [ ]:
Image(cap.stdout)

### Fig. 10B: FlashAttn-v1

For FlashAttn-v1, we can use the official `scaled_dot_product_attention` provided by PyTorch.

To prevent other implementation, please use `sdpa_kernel([SDPBackend.FLASH_ATTENTION])`, we follow the same setup as above:

In [ ]:
%%writefile dmat/fig10b.py
import torch
from neutrino.utils.tensortrace import TensorTrace

torch.manual_seed(42)

with TensorTrace() as t:
    shape = (4, 32, 4096, 64) # batch_size, head_num, seq_len, head_dim
    q = torch.empty(shape, dtype=torch.float16, device="cuda").normal_(mean=0.0, std=0.5)
    k = torch.empty(shape, dtype=torch.float16, device="cuda").normal_(mean=0.0, std=0.5)
    v = torch.empty(shape, dtype=torch.float16, device="cuda").normal_(mean=0.0, std=0.5)
    # dout = torch.randn_like(q) # a random gradient
    causal = False
    sm_scale = 0.5

    with torch.nn.attention.sdpa_kernel([torch.nn.attention.SDPBackend.FLASH_ATTENTION, ]):
        torch.nn.functional.scaled_dot_product_attention(q, k, v, is_causal=causal, scale=sm_scale)

In [ ]:
%%capture cap --no-stderr # use Jupyter Magic to capture the stdout of this cell := path to dmat.png
# Flash Attn Kernels are named as ...flash_fwd_kernel..., let's use flash to locate it
!neutrino -t dmat -k flash python dmat/fig10b.py

In [ ]:
Image(cap.stdout) # display the

### Fig. 10C: Memory Efficient Attention

Similarly, Memory Efficient Attention are also provided via `scaled_dot_product_attention` of PyTorch.

Please use `sdpa_kernel([SDPBackend.EFFICIENT_ATTENTION])` to specify.

In [ ]:
%%writefile dmat/fig10c.py
import torch
from neutrino.utils.tensortrace import TensorTrace

torch.manual_seed(42)

with TensorTrace() as t:
    shape = (4, 32, 4096, 64)
    q = torch.empty(shape, dtype=torch.float16, device="cuda").normal_(mean=0.0, std=0.5)
    k = torch.empty(shape, dtype=torch.float16, device="cuda").normal_(mean=0.0, std=0.5)
    v = torch.empty(shape, dtype=torch.float16, device="cuda").normal_(mean=0.0, std=0.5)
    # dout = torch.randn_like(q) # a random gradient
    causal = False
    sm_scale = 0.5

    with torch.nn.attention.sdpa_kernel([torch.nn.attention.SDPBackend.EFFICIENT_ATTENTION, ]):
        torch.nn.functional.scaled_dot_product_attention(q, k, v, is_causal=causal, scale=sm_scale)

In [ ]:
%%capture cap --no-stderr # use Jupyter Magic to capture the stdout of this cell := path to dmat.png
# Memory Efficient Attention are written in CUTLASS named `fmha`
!neutrino -t dmat -k fmha python dmat/fig10c.py

In [ ]:
Image(cap.stdout) # display the

# Kernel Slowdown and Additional Registers - Part 1

* Correspondence to paper: Table. 2
* Remarks: Benchmarking mode must be enabled through `--benchmark` CLI option. This mode also disable result saving that reduce disk space usage.

Our evaluation consists of three parts:
1. CUTLASS Kernels: Standard GEMM, Stream-K GEMM, Conv2D
2. Triton Kernels: Group MM,  FlashAttn-v2
3. PyTorch Kernels: Batch/LayerNorm, SoftMax, Sum, Max/AvgPool, Embedding, Gather

Evaluation Target:
1. Kenrel Slowdown: `probed / pruned`.
2. Additional Register Usage: `probed - pruned`

In [ ]:
import os
import sys
import subprocess

# METRICS = ["block_sched", "tensorop_count", "gmem_bytes", "dmat"]
METRICS = ["dmat"]
# NOTE filter out distribution kernels (torch.randn) used to initialize test inputs
FILTERED_KERNEL = "distribution:copy"

# PyTorch op_benchmark
TORCH_TESTS = ["norm", "softmax", "sum", "pool", "embedding", "gather"]

for test in TORCH_TESTS:
    os.makedirs(test, exist_ok=True)
    for metric in METRICS:
        os.makedirs(os.path.join(test, metric), exist_ok=True)
        print(test, metric)
        with open(os.path.join(test, metric, "stdout.log"), "w") as f:
            res = subprocess.run(
              ["neutrino", "--benchmark", "-p", f"./{metric}.toml", "--filter", FILTERED_KERNEL, sys.executable, "-m", f"pt.{test}_test"],
              stderr=subprocess.PIPE, stdout=f)
        tracedir = res.stderr.decode().split("\n")[0].strip().split(" ")[-1]
        try:
            os.rename(tracedir, os.path.join(test, metric, "trace"))
        except:
            print(res.stderr.decode())

# Triton and CUTLASS, modified from exposed_latency/main.py
COMMANDS = {
    "attn": [sys.executable, "attn.py",  "4", "32", "4096", "64", "128", "64", "2", "4"],
    "gmm":  [sys.executable, "gmm.py", "512", "512", "512", "512"],
    "gemm": ["../cutlass/build/examples/benchmark/gemm", "--iterations=100"],
    "gemm": ["../cutlass/build/examples/benchmark/gemm-streamk", "--iterations=100"],
    "conv": ["../cutlass/build/examples/benchmark/conv", "--n=256", "--h=56", "--w=56", "--c=128", "--k=128", "--r=3", "--s=3"]
}

for test, command in COMMANDS.items():
    os.makedirs(test, exist_ok=True)
    for metric in METRICS:
        os.makedirs(os.path.join(test, metric), exist_ok=True)
        print(test, metric)
        with open(os.path.join(test, metric, "stdout.log"), "w") as f:
            res = subprocess.run(
                ["neutrino", "--benchmark", "-p", f"./{metric}.toml", "--filter", FILTERED_KERNEL, "-k", test, *command],
                stderr=subprocess.PIPE, stdout=f)
        tracedir = res.stderr.decode().strip().split(" ")[-1]
        os.rename(tracedir, os.path.join(test, metric, "trace"))

## 4. Maximum Memory Usage

* Correspondence to paper: Fig. 11, Sec. 6
* Remarks:
  1. Make sure NO other workload is running on the GPU. (use CUDA_VISIBLE_DEVICES to switch if need)
  2. Here we're not directly evaluating the model but `torch.compile`d modules.
  This is because PyTorch by default use cuBLAS as `nn.Linear` backend but cuBLAS is not tracable by Neutrino (due to dark apis),
  and we can not neglect `nn.Linear`.
  Thus, we use the `torch.compile` to enfore all kernels in PyTorch ATen/Triton for complete tracing and fair evaluation.
  3. Profiling a whole model can be expensive (in time and speed), we recommend first use `--memusage` option to disable the real measurement and only measure the maximum meory usage for a test run.

Evaluation Target: The Maximum Probe Memory Usage

In [ ]:
import os
import shutil
import subprocess

BASE = "max_mem/compiled"
TESTS = ["block_sched", "gmem_bytes", "tensorop_count", "dmat"]
MODELS = ["resnet-bs32", "sd-bs1", "mamba-bs32", "llama1b-bs32", "llama3b-bs32", "llama8b-bs32", "llama8b-bs128", "llama8b-bs256"]

for model in MODELS:
    os.makedirs(os.path.join(model), exist_ok=True)
    for test in TESTS:
        os.makedirs(os.path.join(model, test), exist_ok=True)
        dirs = os.listdir(os.path.join(BASE, model))
        for sub in dirs:
            os.makedirs(os.path.join(model, test, sub[:-3]), exist_ok=True)
            cmd = ["neutrino", "-p", test, "--memusage", "python", os.path.join(BASE, model, sub)]
            print(" ".join(cmd))
            res = subprocess.run(cmd, stderr=subprocess.PIPE, stdout=subprocess.PIPE)
            # print(res)
            for tracedir in res.stderr.decode().split("\n"):
                if len(tracedir) < 3: continue
                # print(tracedir)
                tracedir = tracedir.strip().split(" ")[-1]
                # print(tracedir)
                shutil.copytree(tracedir, os.path.join(model, test, sub[:-3], os.path.basename(tracedir)))
            with open(os.path.join(model, test, sub[:-3], "stdout.log"), "w") as f:
                f.write(res.stdout.decode())

## 5. Exposed Latency

* Correspondence to paper: Fig. 12, Sec. 6
* Remarks:
  1. Make sure NO other workload is running on the GPU. (use CUDA_VISIBLE_DEVICES to switch if need)

In [ ]:
import sys
!{sys.executable} exposed_latency/main.py

## 6. Warp Scheduling and Tailing Effect

* Correspondence to paper: Sec. 7
* Remarks:
  1. Each run of will yield slightly different results
  2. Warp Scheduling varies on different hardware, you will need to use an A100 to have exactly the same results as presented.

In [ ]:
import sys
import os
from IPython.display import Image # For display

In [ ]:
# Run
!cd warp_sched/ && {sys.executable} main.py

In [ ]:
# Fig13a
tracedir = os.path.join("warp_sched", "trace", "exclusive_sched", "result")
trace = os.path.join(tracedir, os.listdir(tracedir)[0])
!{sys.executable} warp_sched/fig13a.py {trace}
Image("warp_sched/fig13a.png")

In [ ]:
# Fig13b
tracedir = os.path.join("warp_sched", "trace", "shared_sched", "result")
trace = os.path.join(tracedir, os.listdir(tracedir)[0])
!{sys.executable} warp_sched/fig13a.py {trace}
Image("warp_sched/fig13b.png")

In [ ]:
# Fig13cd
tracedir = os.path.join("warp_sched", "trace", "shared_acc", "result")
trace = os.path.join(tracedir, os.listdir(tracedir)[0])
!{sys.executable} warp_sched/fig13cd_raw.py {trace}

In [ ]:
Image("warp_sched/fig13c.png")

In [ ]:
Image("warp_sched/fig13d.png")

In [ ]:
Image("warp_sched/fig13d-sub.png")

## 7. Neutrino Internal Workflows

> All results for reproducable are presented above, following contents introduces your Neutrino internal contents to help you judge the Artifact Functionality.

Here we address three question:
1. What is the source code of Neutrino probe to be inserted?
2. How Neutrino raw trace looks like?
3. What is the analyze code to finally print out result?

#### What's the probe code?

We specify the probe with `-p/--probe` CLI option like `-p block_sched`, and these (built-in) probes are under the `tool` folder of Neutrino installation.

In [ ]:
import os
import neutrino # as a python package, Neutrino can be imported!

os.listdir(os.path.join(os.path.dirname(neutrino.__file__), "tools"))

Neutrino takes probes as the `toml` file, and `block_sched` will be automatically matched to the `block_sched.toml` in the above folder.

Moreover, we can further take a look at the content.

In [ ]:
!cat {os.path.join(os.path.dirname(neutrino.__file__), "tools", "block_sched.toml")}

Here you can find the probe body under the `block_sched` section, including four keyword, `position`, `datamodel`, `before` and `after`. More details can be found in Sec. 3.

Moreover, we have some metadata like `analyze_hook` which will be referenced later!

If you want to use your probe, you can use `-p/--probe` and leave your own path to the probe in TOML format!

#### What's the Neutrino Trace?

As a versatile system (Sec. 4), Neutrino trace is not just a simple file and we formulate it into a organized "file system".

First, traces will be placed under `NEUTRINO_TRACEDIR` (configurable via `--trace`), which is default to be `./trace`. By `ls` we can see traces arranged in order:

In [ ]:
os.listdir("block_sched/trace") # take the simplest block_sched as example

Each directory corresponding to traces of a run and it's named by the time and pid.
For example `Apr24_231539_1860576` means it's a run at:
* April 24, 23:15:39, of your local timezone
* PID of this process is 1860576

The use of `pid` is due to the process-independent design of Neutrino that if your program uses multiple process, there'll be multiple trace dir distinguished by the `pid`.

Now we start to explore content of each trace. Probably the latest one.

In [ ]:
# !apt install tree # run this if your system don't have tree
tracedir = os.listdir("block_sched/trace")[-1]
!tree "./trace/{tracedir}" # an undocumented trick

Here you can find:
* `probe.toml`: a copy of probe code shown above, useful for repeating the experiment.
* `event.log`: an important log of all the events captured by Hook Driver (Sec.4.1)
* `kernel`: a folder containing all kernels captured and processed by Neutrino.
* `result`: a folder containing all traces dumped

Now let's deep dive into the `event.log`:

In [ ]:
!cat "block_sched/trace/{tracedir}/event.log"

Let's break it down line to line. **Note that pointers will change from run to run, just for illustration**:
1. `[pid] 1860576` records the pid of this process
2. `[cmd] 26 python block_sched/raw.py` records the command line, useful for classifying traces.
3. `[info] dl 0x26aa350` and `[info] init success` records the status of Hook Driver initialization for internal checking.
4. `[mem]` records the [memory operations](https://docs.nvidia.com/cuda/cuda-driver-api/group__CUDA__MEM.html#group__CUDA__MEM), useful for checking illegal memory access. For example, here it states driver sucessfully (return code 0) allocate 33554432 bytes GMEM at ptr 0x7fdf78000000.
5. `[mod]` records the [module operations](https://docs.nvidia.com/cuda/cuda-driver-api/group__CUDA__MODULE.html#group__CUDA__MODULE), useful for checking GPU code. For example, here it states a function named `_ZN2at6native29vectorized_elementwise_kernel...` was first loaded via `cuLibraryLoadData` from a [fatbinary](https://developer.nvidia.com/blog/cuda-pro-tip-understand-fat-binaries-jit-caching/) and lowered via `cuLibraryGetModule` and `cuModuleGetFunction`.
6. `[exec] funcmap-not-find` states internal function storage don't find the JIT record. This happens for every first-seen code and will trigger JIT probing.
7. `[jit]` records the interaction with probe engines. First it states kernel was `find` from binary storage and was `rename` to SHA1. Next, a folder is created under `kernel` dir and the code was written to `original.bin`. Then a subprocess is forked to launch the probe engine and we wait for the status.
8. `[exec]` records the execution of probing engine.
9. `[analyze]` records the launch details of analyze scripts that finally print out `No.Blocks...

`event.log` records the operations made by the hooked driver, and how about the probe engine?
To address this, we need to explore the contents of `kernel/`.

In [ ]:
!ls "block_sched/trace/{tracedir}/kernel"

Each directory under `kernel/` corresponds to a kernel and is named by the SHA1 of kernel name with an indexed prefix for referencing.

And under each kernel dir, there will be a `original.bin` dumped by hook driver and all the rest are created by hooked driver.
The probe engine follows following steps:
1. `objdump original.bin` to extrace assembly into `original.ptx`. Warning: `original.ptx` can be LARGE because many kernels are fused into one file.
2. Prune the asm by the kernel name (provided as `sys.argv[-1]`, see above `[exec] subproc`) and save to `pruned.ptx` for checking.
3. Probe the asm by the probe in `NEUTRINO_PROBE` envariable and save it to `probed.ptx`
4. `assemble` the `pruned.ptx` and `probed.ptx` into `.bin` of binary machine code.
5. Write the `kernel.info` for the hooked driver to read the enough metadata for `[exec]`.

Moreover, the probed engine will writes all the log into `process.log`, let's take a look first.

In [ ]:
kernel_dir = os.listdir(f"block_sched/trace/{tracedir}/kernel")[0] # will be 0_xxx
!cat "block_sched/trace/{tracedir}/kernel/{kernel_dir}/process.log"

Let's break it from top to down:
* First line is the kernel filtering information (set via `--filter`/`--kernel`). It's empty here because we don't set anything.
* Second line is the `objdump` information, stating the command used to dump the code.
* Third line is the kernel name being used.
* Then it's the command and logs of assembler (`ptxas` for NVIDIA). Please pay special attention to the last line stating no.registers used (for producing Table. 2) and constant memroy `cmem[0]` (used for kernel parameters, here Neutrino use 8 bytes more for a 64bit pointer).
* Finally is the auto-generated Python code for trace reading conforming Sec. 4.4, helpful for building trace analysis tools, detailed presented in answering the 3rd question.

And when the probing engine fails, it will also print out the trace back tree here for analysis, like this:
```
['']
[decompile] via cuobjdump -ptx
_ZN2at6native29vectorized_elementwise_kernelILi4ENS0_11FillFunctorIN3c104HalfEEENS_6detail5ArrayIPcLi1EEEEEviT0_T1_
Traceback (most recent call last):
  File "/home/root/anaconda3/envs/testenv/lib/python3.11/site-packages/neutrino/probe/cuda.py", line 747, in <module>
    probed_ptx, probe_mem_sizes, trace_reading_code = probing(entry_section, probes)
                                                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/root/anaconda3/envs/testenv/lib/python3.11/site-packages/neutrino/probe/cuda.py", line 507, in probing
    in3:  str = operands[3] if len(operands) >= 4 else None
    ^^^
UnboundLocalError: cannot access local variable 'out' where it is not associated with a value
```

We build the probe engine in Python so you can easily debug and extend new functionalities if the current cannot fulfill your need.

Moreover, `process.log` is for huamn-reading, another log for the hook driver to read back is the `kernel.info`.

In [ ]:
!cat "block_sched/trace/{tracedir}/kernel/{kernel_dir}/kernel.info"

From top to down, `kernel.info` contains:
* First line is the kernel name, like `_ZN2at6native29vectorized_elementwise_kernel...`
* Second line is the number of original kernel parameters, like 3.
* Third line is the number of probes *saving records*, here is 1. Then it's `n=1` lines followed containing the datamodel, of type (`0:=thread/1:=warp`) and bytes, 16 here, saved for each thread/warp.
* Later, there'll be a line containing optional analyze hook, like `block_sched.py` here.

These will be parsed by the hook driver for execution usage (`[exec]`).

Finally, we can take a look at the difference of `pruned.ptx` and `probed.ptx`:

In [ ]:
!cat "block_sched/trace/{tracedir}/kernel/{kernel_dir}/pruned.ptx"

In [ ]:
!cat "block_sched/trace/{tracedir}/kernel/{kernel_dir}/probed.ptx"

Here we can the global definition are kept and the probe engine make following modification:
1. A parameter `.param .u64 param_block_sched` is added
2. Probes added at corresponding places, i.e., kernel start and end
3. Buffer calculation in `// begin buffer calculation` and `// begin block_sched buffer`

#### What is the analyze code to finally print out result?

Upon here, the last question is how the `No.block:32768 Running:602241 Scheduling:87713(cycle)` is analyzed and printed.

First, Neutrino will save all raw traces in the `result` directory.
Raw traces are of raw binary and is orderly named by the TIME since the driver starts.

In [ ]:
!ls "block_sched/trace/{tracedir}/result/"

To convert the raw binary into valuable analyzed results, Neutrino provides analyze_hook for user to register in the `probe.toml`.
For example, here we register the `block_sched.py` as the analyze hook. Relative paths will be resolved based on `tools/` directory of installation folder.

In [ ]:
!cat "block_sched/trace/{tracedir}/probe.toml"

In [ ]:
# Real code of block_sched.py
!cat {os.path.join(os.path.dirname(neutrino.__file__), "tools", "block_sched.py")}

There are two part of the code, separated by `# END OF GENERATED CODE`:
* 1st part is the trace reading code auto-generated by Neutrino (see `process.log` above) that are used to read `.bin` traces into Python objects familiar to developers.
* 2nd part is the analyze code built upon the `parse` written by developers. Here we simulate a FIFO scheduler based on traces collected, and print and calculate the scheduling times.